In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
n_users = 74
gpu_id = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
# Download the dataset
download_elec_dataset()

# Clean dataset
elec_data = clean_elec()

from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(elec_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(elec_data)

elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

Files are ready
Data loaded..
Data aggregated by hour: (26304, 370)


In [6]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# elec_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/electricity/electricity.csv', index_label='date')


In [7]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(elec_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.data_path

'electricity.csv'

In [8]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [9]:
test_data[0][0].shape

(96, 371)

In [10]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [11]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(5, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(5, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
            

# Setup WANDB

In [12]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Elec dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Elec dataset",
    # Hyperparams
    config={
        "dataset": "Elec370",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 370,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [13]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 33.48630714416504
Epoch: 2 cost time: 34.29861617088318
Epoch: 3 cost time: 32.09459185600281
Epoch: 4 cost time: 32.078121185302734
Epoch: 5 cost time: 32.50928020477295
Epoch: 1 cost time: 32.27593159675598
Epoch: 2 cost time: 31.791346788406372
Epoch: 3 cost time: 39.77666783332825
Epoch: 4 cost time: 44.17271280288696
Epoch: 5 cost time: 44.25560760498047
Epoch: 1 cost time: 44.680445194244385
Epoch: 2 cost time: 43.66454315185547
Epoch: 3 cost time: 52.58669662475586
Epoch: 4 cost time: 55.22741150856018
Epoch: 5 cost time: 43.71111989021301
Epoch: 1 cost time: 45.67724108695984
Epoch: 2 cost time: 45.00049018859863
Epoch: 3 cost time: 44.529595136642456
Epoch: 4 cost time: 44.15816283226013
Epoch: 5 cost time: 44.969921827316284
Epoch: 1 cost time: 45.89774703979492
Epoch: 2 cost time: 44.07857131958008
Epoch: 3 cost time: 45.04936981201172
Epoch: 4 cost time: 44.20404052734375
Epoch: 5 cost time: 44.34987425804138
Epoch: 1 cost time: 44.61495876312256
Epoch: 

Progress:   3%|▎         | 1/30 [59:03<28:32:43, 3543.58s/it]

Epoch: 1 cost time: 335.1749963760376
Epoch: 2 cost time: 335.1530976295471
Epoch: 3 cost time: 335.03712487220764
Epoch: 4 cost time: 338.1483292579651
Epoch: 5 cost time: 333.73820543289185
Epoch: 1 cost time: 333.5439417362213
Epoch: 2 cost time: 334.6799461841583
Epoch: 3 cost time: 337.0410599708557
Epoch: 4 cost time: 335.34168004989624
Epoch: 5 cost time: 333.9395604133606
Epoch: 1 cost time: 335.2722156047821
Epoch: 2 cost time: 333.656188249588
Epoch: 3 cost time: 334.5750858783722
Epoch: 4 cost time: 334.399126291275
Epoch: 5 cost time: 336.19660663604736
Epoch: 1 cost time: 336.04306173324585
Epoch: 2 cost time: 334.3306562900543
Epoch: 3 cost time: 334.57905626296997
Epoch: 4 cost time: 333.74597454071045
Epoch: 5 cost time: 333.7643551826477
Epoch: 1 cost time: 332.43486046791077
Epoch: 2 cost time: 335.8536763191223
Epoch: 3 cost time: 335.1887710094452
Epoch: 4 cost time: 333.4317467212677
Epoch: 5 cost time: 334.8606150150299
Epoch: 1 cost time: 335.5878052711487
Epoch:

Progress:   7%|▋         | 2/30 [4:26:20<68:14:56, 8774.88s/it]

...

Progress:  93%|█████████▎| 28/30 [42:29:06<55:55, 1677.90s/it]  

Epoch: 1 cost time: 41.33475947380066
Epoch: 2 cost time: 41.46291136741638
Epoch: 3 cost time: 41.59420871734619
Epoch: 4 cost time: 41.14297580718994
Epoch: 5 cost time: 41.010048627853394
Epoch: 1 cost time: 41.280059576034546
Epoch: 2 cost time: 40.86799240112305
Epoch: 3 cost time: 41.52587676048279
Epoch: 4 cost time: 41.765283823013306
Epoch: 5 cost time: 41.22102165222168
Epoch: 1 cost time: 41.14507579803467
Epoch: 2 cost time: 40.98021674156189
Epoch: 3 cost time: 41.456870555877686
Epoch: 4 cost time: 41.05886459350586
Epoch: 5 cost time: 41.22269868850708
Epoch: 1 cost time: 41.140334129333496
Epoch: 2 cost time: 40.901944160461426
Epoch: 3 cost time: 41.30140686035156
Epoch: 4 cost time: 43.00628161430359
Epoch: 5 cost time: 42.621697664260864
Epoch: 1 cost time: 43.21843504905701
Epoch: 2 cost time: 43.51136803627014
Epoch: 3 cost time: 42.83880567550659
Epoch: 4 cost time: 41.65957593917847
Epoch: 5 cost time: 41.2085542678833
Epoch: 1 cost time: 41.562633752822876
Epoch

Progress:  97%|█████████▋| 29/30 [42:54:42<27:15, 1635.53s/it]

Epoch: 1 cost time: 41.152053117752075
Epoch: 2 cost time: 41.45941185951233
Epoch: 3 cost time: 41.4472599029541
Epoch: 4 cost time: 41.549192667007446
Epoch: 5 cost time: 42.041605949401855
Epoch: 1 cost time: 43.12444305419922
Epoch: 2 cost time: 43.80986928939819
Epoch: 3 cost time: 42.91979789733887
Epoch: 4 cost time: 42.79918813705444
Epoch: 5 cost time: 42.53087854385376
Epoch: 1 cost time: 41.47823429107666
Epoch: 2 cost time: 41.42333483695984
Epoch: 3 cost time: 41.464150190353394
Epoch: 4 cost time: 41.24242281913757
Epoch: 5 cost time: 41.769294023513794
Epoch: 1 cost time: 41.161458253860474
Epoch: 2 cost time: 41.24021077156067
Epoch: 3 cost time: 41.22733521461487
Epoch: 4 cost time: 41.65654921531677
Epoch: 5 cost time: 41.32395076751709
Epoch: 1 cost time: 41.341915130615234
Epoch: 2 cost time: 40.84609508514404
Epoch: 3 cost time: 41.17528414726257
Epoch: 4 cost time: 41.281386852264404
Epoch: 5 cost time: 40.98345327377319
Epoch: 1 cost time: 41.02763628959656
Epoch

Progress: 100%|██████████| 30/30 [43:20:20<00:00, 5200.67s/it]
